## ライブラリのインポート

In [ ]:
import numpy as np
import pandas as pd

import pandas as pd
import MeCab
import re
import tqdm

import matplotlib.pyplot as plt

from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel

# LDAの可視化
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

import warnings
warnings.simplefilter('ignore')

## DataFrame型の変換

In [ ]:
df = pd.read_csv("text/livedoor.tsv", sep='\t')

# categoryを1つのカラムにまとめる
df = pd.melt(df,id_vars=df.columns.values[:2],var_name="category",value_name="num" )
df = df[df["num"]==1][["category", "article"]]
df.head()

,category,article
0,dokujo-tsushin,もうすぐジューン・ブライドと呼ばれる６月。独女の中には自分の式はまだなのに呼ばれてばかり…...
1,dokujo-tsushin,携帯電話が普及する以前、恋人への連絡ツールは一般電話が普通だった。恋人と別れたら、手帳に書...
2,dokujo-tsushin,「男性はやっぱり、女性の“すっぴん”が大好きなんですかね」と不満そうに話すのは、出版関係で...
3,dokujo-tsushin,ヒップの加齢による変化は「たわむ→下がる→内に流れる」、バストは「そげる→たわむ→外に流れ...
4,dokujo-tsushin,6月から支給される子ども手当だが、当初は子ども一人当たり月額2万6000円が支給されるはず...


## 前処理

In [ ]:
# ユニコード正規化
df["article"] = df["article"].str.normalize("NFKC")
# アルファベットを小文字に統一
df["article"] = df["article"].str.lower()

In [ ]:
# 分かち書きの中で使うオブジェクト生成
tagger = MeCab.Tagger() # mecab-ipadic-neologd のパス
# ひらがなのみの文字列にマッチする正規表現
kana_re = re.compile("^[ぁ-ゖ]+$")

In [ ]:
def mecab_tokenizer(text):
    # テキストを分かち書きする関数を準備する
    parsed_lines = tagger.parse(text).split("\n")[:-2]
    surfaces = [l.split('\t')[0] for l in parsed_lines]
    features = [l.split('\t')[1] for l in parsed_lines]
    # 原型を取得
    bases = [f.split(',')[6] for f in features]
    # 品詞を取得
    pos = [f.split(',')[0] for f in features]

    # 各単語を原型に変換する
    token_list = [b if b != '*' else s for s, b in zip(surfaces, bases)]

    # 名詞,動詞,形容詞のみに絞り込み
    target_pos = ["名詞", "動詞", "形容詞"]
    token_list = [t for t, p in zip(token_list, pos) if p in target_pos]

    # アルファベットを小文字に統一
    token_list = [t.lower() for t in token_list]

    # ひらがなのみの単語を除く
    token_list = [t for t in token_list if not kana_re.match(t)]

    return token_list

In [ ]:
# 分かち書きしたデータを作成する
sentences = df["article"].apply(mecab_tokenizer)

In [ ]:
print(sentences[:5])

0    [ジューン・ブライド, 呼ぶ, 6月, 独女, 中, 自分, 式, 呼ぶ, お祝い, 貧乏,...
1    [携帯電話, 普及, 以前, 恋人, 連絡, ツール, 一般, 電話, 普通, 恋人, 別れ...
2    [男性, 女性, 大好き, 不満, 話す, 出版, 関係, 働く, 香, 仮名, 31歳, ...
3    [ヒップ, 加, 齢, 変化, 下がる, 内, 流れる, バスト, 外, 流れる, バスト,...
4    [6月, 支給, 子ども手当, 当初, 子ども, 一, 人, 当たり, 月額, 2, 万, ...
Name: article, dtype: object


In [ ]:
# 単語と単語IDを対応させる辞書の作成
dictionary = Dictionary(sentences)

In [ ]:
# LdaModelが読み込めるBoW形式に変換
corpus = [dictionary.doc2bow(text) for text in sentences]

In [ ]:
# 5000番目のテキストを変換した結果。(長いので10単語で打ち切って表示)
print(corpus[5000][:10])

[(30, 2), (46, 1), (65, 7), (73, 5), (76, 1), (105, 1), (112, 1), (123, 1), (124, 1), (170, 1)]


In [ ]:
# idから単語を取得
print(dictionary[119])

自分


In [ ]:
# 単語からidを取得
print(dictionary.token2id["復帰"])

8635


## LDAによるトピックモデル

In [ ]:
# トピック数を指定してモデルを学習
lda = LdaModel(corpus, num_topics=9)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [ ]:
# 学習したモデルを使って、テキストをトピックスに変換
print(lda[corpus[0]])

[(0, 0.044198893), (2, 0.07959694), (3, 0.7357896), (4, 0.13784097)]


In [ ]:
# DataFrameに変換
topic_df = pd.DataFrame(index=range(len(corpus)))

In [ ]:
for c in range(9):
    topic_df[c] = 0.0

In [ ]:
for i in range(len(corpus)):
    topics = lda[corpus[i]]
    for t, p in  topics:

        topic_df.loc[i][t] = p

In [ ]:
topic_df.head().round(3)

,0,1,2,3,4,5,6,7,8
0,0.044,0.000,0.080,0.735,0.139,0.000,0.00,0.0,0.000
1,0.000,0.610,0.000,0.000,0.326,0.000,0.06,0.0,0.000
2,0.000,0.131,0.000,0.000,0.440,0.137,0.00,0.0,0.289
3,0.000,0.000,0.156,0.000,0.317,0.000,0.48,0.0,0.045
4,0.030,0.799,0.000,0.169,0.000,0.000,0.00,0.0,0.000


In [ ]:
len(topic_df)

7367

## 元のカテゴリーとTopicの対応

In [ ]:
main_topic = topic_df.values.argmax(axis=1)

In [ ]:
main_topic

array([3, 1, 4, ..., 7, 1, 3])

## トピックごとの出現頻度上位の単語のIDとその確率

In [ ]:
# 空のDataFrameを作成
df_topic = pd.DataFrame(
    [
     [],
     [],
     []
    ]
).T

# ID、トピック、確率をDataFrameに格納
for i, prob in lda.get_topic_terms(2, topn=20):
    df_topic = pd.concat([df_topic, pd.DataFrame([i, dictionary.id2token[int(i)], round(prob, 3)]).T])

df_topic.columns = [["ID", "トピック", "確率"]]
df_topic = df_topic.reset_index()[["ID", "トピック", "確率"]]
df_topic

,ID,トピック,確率
0,22397,d,0.012
1,73,思う,0.007
2,33,人,0.006
3,5582,s,0.005
4,15821,sh,0.005
5,25349,ソフトバンク,0.005
6,65,対応,0.004
7,119,自分,0.004
8,18687,xi,0.004
9,29926,ロンドン五輪,0.004


## テキストに対応するトピックを表示

In [ ]:
total_topic_contents_list = []
rank = 3

for i in tqdm.tqdm_notebook(range(len(corpus))):
    topic_contents_list = []                                                                # コーパスの各行ごとのトピックだけを格納するため初期化

    toipc_ID_prob_list = np.array(lda[corpus[i]])                               # 各行のコーパスに対するトピックIDとその確率
    np_toipc_prob = np.array(toipc_ID_prob_list)[:, 1]                     # スライスで確率のみ抽出
    num_of_slices = min(rank, len( toipc_ID_prob_list))                  # 上位3件または要素数の小さい方 要素数が2以下の場合の対策
    first_thirrd_list = np.argpartition(-np_toipc_prob, num_of_slices -1)[:num_of_slices]    # 上位3件の確率のインデックスを取得

    for i in range(len(first_thirrd_list)): # 上位3件のトピックを topic_contents_listに格納
        topic_contents_list.append(dictionary.id2token[int(toipc_ID_prob_list[:, 0][first_thirrd_list[i]])])
    
    total_topic_contents_list.append(topic_contents_list)                # 各行のコーパスを1つの要素として格納するリスト

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [ ]:
df["topic"] =  total_topic_contents_list
df = df[["category", "topic", "article"]]
df.head()

,category,topic,article
0,dokujo-tsushin,"[33歳, 35歳, 30歳]",もうすぐジューン・ブライドと呼ばれる6月。独女の中には自分の式はまだなのに呼ばれてばかり....
1,dokujo-tsushin,"[10万, 35歳, 6月]",携帯電話が普及する以前、恋人への連絡ツールは一般電話が普通だった。恋人と別れたら、手帳に書...
2,dokujo-tsushin,"[35歳, お願い, 3人]",「男性はやっぱり、女性の“すっぴん”が大好きなんですかね」と不満そうに話すのは、出版関係で...
3,dokujo-tsushin,"[6月, 35歳, 30歳]",ヒップの加齢による変化は「たわむ→下がる→内に流れる」、バストは「そげる→たわむ→外に流れ...
4,dokujo-tsushin,"[10万, 33歳, 1000]",6月から支給される子ども手当だが、当初は子ども一人当たり月額2万6000円が支給されるはず...


## LDAモデルの可視化

### 可視化1（PCoA）

In [ ]:
vis = pyLDAvis.gensim_models.prepare(lda, corpus, dictionary, n_jobs = 1, sort_topics = False)

pyLDAvis.display(vis)

### 可視化2（Metric MDS）

In [ ]:
vis =pyLDAvis.gensim_models.prepare(lda, corpus, dictionary, n_jobs = 1, mds='mmds', sort_topics = False)

pyLDAvis.display(vis)